## ERA5 Zarr数据说明

- 1. <https://home.chpc.utah.edu/~u0553130/Brian_Blaylock/cgi-bin/generic_AWS_download.cgi?DATASET=era5-pds&BUCKET=zarr/2023/08/data>
- 2. <https://registry.opendata.aws/ecmwf-era5/>
 
```bash
# scoop install aws
aws s3 ls --no-sign-request s3://era5-pds/zarr/2023/08/data/

PRE air_pressure_at_mean_sea_level.zarr/
PRE air_temperature_at_2_metres.zarr/
PRE air_temperature_at_2_metres_1hour_Maximum.zarr/
PRE air_temperature_at_2_metres_1hour_Minimum.zarr/
PRE dew_point_temperature_at_2_metres.zarr/
PRE eastward_wind_at_100_metres.zarr/
PRE eastward_wind_at_10_metres.zarr/
PRE integral_wrt_time_of_surface_direct_downwelling_shortwave_flux_in_air_1hour_Accumulation.zarr/
PRE lwe_thickness_of_surface_snow_amount.zarr/
PRE northward_wind_at_100_metres.zarr/
PRE northward_wind_at_10_metres.zarr/
PRE precipitation_amount_1hour_Accumulation.zarr/
PRE sea_surface_temperature.zarr/
PRE snow_density.zarr/
PRE surface_air_pressure.zarr/
```

In [49]:
vars = [
	"air_pressure_at_mean_sea_level.zarr/", 
	"air_temperature_at_2_metres.zarr/", 
	"air_temperature_at_2_metres_1hour_Maximum.zarr/", 
	"air_temperature_at_2_metres_1hour_Minimum.zarr/", 
	"dew_point_temperature_at_2_metres.zarr/", 
	"eastward_wind_at_100_metres.zarr/", 
	"eastward_wind_at_10_metres.zarr/", 
	"integral_wrt_time_of_surface_direct_downwelling_shortwave_flux_in_air_1hour_Accumulation.zarr/", 
	"lwe_thickness_of_surface_snow_amount.zarr/", 
	"northward_wind_at_100_metres.zarr/", 
	"northward_wind_at_10_metres.zarr/", 
	"precipitation_amount_1hour_Accumulation.zarr/", 
	"sea_surface_temperature.zarr/", 
	"snow_density.zarr/", 
	"surface_air_pressure.zarr/"
]
# 提供了15个变量的在线访问与下载

15-element Vector{String}:
 "air_pressure_at_mean_sea_level.zarr/"
 "air_temperature_at_2_metres.zarr/"
 "air_temperature_at_2_metres_1hour_Maximum.zarr/"
 "air_temperature_at_2_metres_1hour_Minimum.zarr/"
 "dew_point_temperature_at_2_metres.zarr/"
 "eastward_wind_at_100_metres.zarr/"
 "eastward_wind_at_10_metres.zarr/"
 "integral_wrt_time_of_surface_di" ⋯ 32 bytes ⋯ "in_air_1hour_Accumulation.zarr/"
 "lwe_thickness_of_surface_snow_amount.zarr/"
 "northward_wind_at_100_metres.zarr/"
 "northward_wind_at_10_metres.zarr/"
 "precipitation_amount_1hour_Accumulation.zarr/"
 "sea_surface_temperature.zarr/"
 "snow_density.zarr/"
 "surface_air_pressure.zarr/"

# 1. Zarr

In [27]:
using Zarr, YAXArrays
include("main_cube.jl")

chunksize (generic function with 1 method)

In [11]:
f = "https://era5-pds.s3.amazonaws.com/zarr/2023/08/data/air_pressure_at_mean_sea_level.zarr"
# 只能一个月，一个变量的方式进行下载数据
z = zopen(f, consolidated=true)

ZarrGroup at Consolidated Consolidated Zarr.HTTPStore("https://era5-pds.s3.amazonaws.com/zarr/2023/08/data/air_pressure_at_mean_sea_level.zarr") and path 
Variables: lat time0 air_pressure_at_mean_sea_level lon 

In [13]:
# z["air_pressure_at_mean_sea_level"]
z.arrays

Dict{String, ZArray} with 4 entries:
  "lat"                            => ZArray{Float32} of size 721
  "time0"                          => ZArray{Float64} of size 744
  "air_pressure_at_mean_sea_level" => ZArray{Float32} of size 1440 x 721 x 744
  "lon"                            => ZArray{Float32} of size 1440

In [ ]:
lat = z["lat"][:]
lon = z["lon"][:]

In [21]:
function clip_region(lon, lat, range = [70, 140, 15, 55])
    ilon = findall(@.(range[1] <= lon <= range[2]))
    ilon = ilon[1]:ilon[end]
    ilat = findall(@.(range[3] <= lat <= range[4]))
    ilat = ilat[1]:ilat[end]
    ilon, ilat
end

ilon, ilat = clip_region(lon, lat)
# 获取中国的数据
@time data = z["air_pressure_at_mean_sea_level"][ilon, ilat, :];
# 自己构建YAXArrays速度可能会稍微快一点

 17.003729 seconds (4.97 M allocations: 900.857 MiB, 1.49% gc time, 12.92% compilation time)


In [23]:
## 然后是如何保存数据
using Ipaper
obj_size(data)

[ Info: Precompiling Ipaper [e58298cb-69f7-4186-aecd-5834b6793426]
[ Info: Precompiling IpaperSlopeExt [3187b56e-9583-5004-b811-c64661423187]


Array{Float32, 3} | (281, 161, 744) | 128.4 Mb


In [24]:
f = "https://era5-pds.s3.amazonaws.com/zarr"
z = zopen(f)

LoadError: ArgumentError: Could not find consolidated metadata for store Zarr.HTTPStore("https://era5-pds.s3.amazonaws.com/zarr")

# 2. YAXArrays

In [26]:
f = "https://era5-pds.s3.amazonaws.com/zarr/"
f = "https://era5-pds.s3.amazonaws.com/zarr/2023/08/data/air_pressure_at_mean_sea_level.zarr"
@time ds = open_dataset(f)

  7.720878 seconds (5.24 M allocations: 342.242 MiB, 0.65% gc time, 74.22% compilation time: 8% of which was recompilation)


YAXArray Dataset
Shared Axes: 
Dim{:lon} Sampled{Float32} 0.0f0:0.25f0:359.75f0 ForwardOrdered Regular Points,
Dim{:lat} Sampled{Float32} 90.0f0:-0.25f0:-90.0f0 ReverseOrdered Regular Points,
Ti Sampled{DateTime} DateTime[2023-08-01T00:00:00, …, 2023-08-31T23:00:00] ForwardOrdered Irregular Points
Variables: 
air_pressure_at_mean_sea_level, 
Properties: Dict{String, Any}("source" => "Reanalysis", "title" => "ERA5 forecasts", "institution" => "ECMWF")


In [35]:
@time cube = ds[1][ilon, ilat, :]

 16.483381 seconds (186.46 k allocations: 616.133 MiB, 0.20% gc time)


281×161×744 YAXArray{Float32,3} with dimensions: 
  Dim{:lon} Sampled{Float32} 70.0f0:0.25f0:140.0f0 ForwardOrdered Regular Points,
  Dim{:lat} Sampled{Float32} 55.0f0:-0.25f0:15.0f0 ReverseOrdered Regular Points,
  Ti Sampled{DateTime} DateTime[2023-08-01T00:00:00, …, 2023-08-31T23:00:00] ForwardOrdered Irregular Points
units: Pa
name: air_pressure_at_mean_sea_level
Total size: 128.4 MB


In [36]:
# 然后是保存数据
@time savecube(ds[1][ilon, ilat, :], "datacube-ERA5/Pa")

 22.352102 seconds (187.22 k allocations: 1.163 GiB, 2.50% gc time)


281×161×744 YAXArray{Float32,3} with dimensions: 
  Dim{:lon} Sampled{Float32} 70.0f0:0.25f0:140.0f0 ForwardOrdered Regular Points,
  Dim{:lat} Sampled{Float32} 55.0f0:-0.25f0:15.0f0 ReverseOrdered Regular Points,
  Ti Sampled{DateTime} DateTime[2023-08-01T00:00:00, …, 2023-08-31T23:00:00] ForwardOrdered Irregular Points
units: Pa
name: air_pressure_at_mean_sea_level
Total size: 128.4 MB


In [37]:
@time begin
    c = ds[1][ilon, ilat, :]
    savecube(c, "datacube-ERA5/Pa-02")
end

 18.771287 seconds (166.43 k allocations: 1.167 GiB, 3.38% gc time)


281×161×744 YAXArray{Float32,3} with dimensions: 
  Dim{:lon} Sampled{Float32} 70.0f0:0.25f0:140.0f0 ForwardOrdered Regular Points,
  Dim{:lat} Sampled{Float32} 55.0f0:-0.25f0:15.0f0 ReverseOrdered Regular Points,
  Ti Sampled{DateTime} DateTime[2023-08-01T00:00:00, …, 2023-08-31T23:00:00] ForwardOrdered Irregular Points
units: Pa
name: air_pressure_at_mean_sea_level
Total size: 128.4 MB


In [48]:
# Zarr.AWSS3.AWS.global_aws_config(Zarr.AWSS3.AWS.AWSConfig(creds=nothing)) # region="eu-west-2"

year = 2023
month = @sprintf("%02d", 8)

vars = ["air_temperature_at_2_metres",
   "dew_point_temperature_at_2_metres",
   "air_pressure_at_mean_sea_level",
   "northward_wind_at_10_metres",
   "eastward_wind_at_10_metres"]
var = vars[1]
f = "s3://era5-pds/zarr/$year/$month/data/$var.zarr/"

@time ds = open_dataset(f)

  5.083435 seconds (9.99 k allocations: 786.188 KiB)


YAXArray Dataset
Shared Axes: 
Dim{:lon} Sampled{Float32} 0.0f0:0.25f0:359.75f0 ForwardOrdered Regular Points,
Dim{:lat} Sampled{Float32} 90.0f0:-0.25f0:-90.0f0 ReverseOrdered Regular Points,
Ti Sampled{DateTime} DateTime[2023-08-01T00:00:00, …, 2023-08-31T23:00:00] ForwardOrdered Irregular Points
Variables: 
air_temperature_at_2_metres, 
Properties: Dict{String, Any}("source" => "Reanalysis", "title" => "ERA5 forecasts", "institution" => "ECMWF")


In [51]:
@time savecube(ds[1][ilon, ilat, :], "datacube-ERA5/air_temperature")

 41.556399 seconds (136.82 k allocations: 1.390 GiB, 1.77% gc time)


281×161×744 YAXArray{Float32,3} with dimensions: 
  Dim{:lon} Sampled{Float32} 70.0f0:0.25f0:140.0f0 ForwardOrdered Regular Points,
  Dim{:lat} Sampled{Float32} 55.0f0:-0.25f0:15.0f0 ReverseOrdered Regular Points,
  Ti Sampled{DateTime} DateTime[2023-08-01T00:00:00, …, 2023-08-31T23:00:00] ForwardOrdered Irregular Points
units: K
name: air_temperature_at_2_metres
Total size: 128.4 MB


In [55]:
# ds.GridChunks
chunksize(ds[1]) # 如果调整chunksize，速度应该会更快

(150, 150, 372)